# Dillards POS Data Association Rules Project

    Charlie Marshall
    IEMS 308
    13 February 2020

## Loading Relevant Packages

In [1]:
import os
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## Loading Relevant Data Sets from Dillards POS Data

In [2]:
deptinfo = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/deptinfo.csv",
                       sep = ',', names = ('Dept','Description','Zeros'))
deptinfo = deptinfo.drop('Zeros', axis = 1)

In [3]:
deptinfo.head()

Dept Description
0   800    CLINIQUE
1   801    LESLIE  
2  1100    GARY F  
3  1107    JACQUES 
4  1202    CABERN

In [4]:
deptinfo.shape

(60, 2)

In [5]:
deptinfo[deptinfo['Dept'] == 6505]

Dept Description
41  6505    ST JOHN

In [6]:
skuinfo = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/skuinfo.csv",
                      sep = ',', usecols = [0,1,2,3,4,5,6,7,8,9],
                      names = ('SKU', 'Dept', 'ClassID', 'UPC', 'Style', 'Color', 'Size', 'Packsize', 'Vendor', 'Brand'),
                     dtype={'Packsize': str, 'Vendor': str})

In [7]:
skuinfo.head()

SKU  Dept ClassID           UPC         Style         Color        Size  \
0    3  6505     113  400000003000  00    F55KT2  WHISPERWHITE  P8EA         
1    4  8101     002  400000004000  22    615CZ4  SPEARMI       S            
2    5  7307     003  400000005000  7LBS  245-01  34 SILVER     KING         
3    8  3404     00B  400000008000  622   F05H84  MORNING MI    2T           
4   15  2301     004  400000015000  126   MDU461  255CAMEL      12           

  Packsize   Vendor      Brand  
0        1  5119207  TURNBURY   
1        1  3311144  C A SPOR   
2        1  5510554  BEAU IDE   
3        1  2912827  HARTSTRI   
4        1  0023272  JONES/LA

In [8]:
skuinfo.shape

(1564178, 10)

In [9]:
len(set(skuinfo['SKU']))

1564178

In [10]:
skstinfo = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/skstinfo.csv",
                       sep = ',', names = ('SKU', 'Store', 'Cost', 'Retail', 'Zeros'))

In [11]:
skstinfo = skstinfo.drop('Zeros', axis = 1)

In [12]:
skstinfo.head()

SKU  Store    Cost  Retail
0    3    102  123.36   440.0
1    3    103  123.36   440.0
2    3    104  123.36   440.0
3    3    202  123.36   440.0
4    3    203  123.36   440.0

In [13]:
skstinfo.shape

(39230146, 4)

In [14]:
strinfo = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/strinfo.csv",
                      sep = ',', names = ('Store','City','State','Zip','Zeros'))

In [15]:
strinfo = strinfo.drop('Zeros', axis = 1)

In [16]:
strinfo.head()

Store                  City State    Zip
0      2  ST. PETERSBURG          FL  33710
1      3  ST. LOUIS               MO  63126
2      4  LITTLE ROCK             AR  72201
3      7  FORT WORTH              TX  76137
4      9  TEMPE                   AZ  85281

In [17]:
strinfo.shape

(453, 4)

The trnsact data set is extremely large and cannot be loaded in completely. In order to combat this I only used columns which might be relevant in further stages of analysis.

In [18]:
trnsact = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/trnsact.csv",
                      na_filter = False, usecols = [0,1,2,3,4,5,6],
                      names = ('SKU','Store','Register','TranNum','SEQ','Date','SType'))

In [19]:
trnsact.head()

SKU  Store  Register  TranNum        SEQ        Date SType
0    3    202       290     1100  326708721  2005-01-18     P
1    3    202       540     2700  326708721  2005-01-29     R
2    3    303       500     2100   23702074  2004-08-18     P
3    3    709       360      500          0  2005-08-14     P
4    3    802       660      400          0  2005-08-09     P

In [20]:
trnsact.shape

(120916896, 7)

In [21]:
trnsact['Date'] = pd.to_datetime(trnsact['Date'])

In [22]:
trnsact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120916896 entries, 0 to 120916895
Data columns (total 7 columns):
SKU         int64
Store       int64
Register    int64
TranNum     int64
SEQ         int64
Date        datetime64[ns]
SType       object
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 6.3+ GB


## Preparing Data for Association Rules Analysis - Subsetting

Taking data from raw form to the point where it is ready to be analysed with association rules. This includes making decisions about subsetting the data. I decided to subset the data by only analyzing certain departments at a time. Additionally, we only want to look at purchase data, not return data.

### Subset the data to only include SKUs from the Polo Men's Department

In [23]:
deptinfo[deptinfo['Description'] == 'POLOMEN ']

Dept Description
25  4505    POLOMEN

In [24]:
deptID = 4505

Notice that the Deptartment number is 4505 for Polo Men

In [25]:
poloSKU = skuinfo[skuinfo['Dept'] == deptID]

In [26]:
poloSKU.head()

SKU  Dept ClassID           UPC         Style         Color        Size  \
52  210  4505     107  400000210000  9ALDN 797442  FUCHSIA/FO    160          
60  258  4505     107  400000258000  9A    797493  BANANA BEA    XXL          
71  308  4505     424  400000308000  1BR   580599  CLSC WASH     36           
83  367  4505     109  400000367000  9AME  096278  BLUE/NAVY     1654         
88  397  4505     108  400000397000  8     047338  ANDOVER HE    M            

   Packsize   Vendor      Brand  
52        1  5715232  POLO FAS   
60        1  5715232  POLO FAS   
71        1  5715232  POLO FAS   
83        1  5715232  POLO FAS   
88        1  5715232  POLO FAS

In [27]:
# The number of unique SKUs associated with the data
len(set(poloSKU['SKU']))

142108

Subsetting trnsact data using only the Polo Men Department

In [28]:
poloData = trnsact[trnsact['SKU'].isin(skuinfo[skuinfo['Dept'] == deptID]['SKU'])]

In [29]:
poloData.head()

SKU  Store  Register  TranNum        SEQ       Date SType
582    210   7003       180     4000          0 2004-09-13     P
937    367   2707       370    14200          0 2004-10-01     P
938    367   3307       411     3100          0 2004-10-02     P
1618   397    303       910     6100  544100323 2004-08-21     P
5055  1165    107        31     4300          0 2004-09-05     P

In [30]:
poloData.shape

(5763323, 7)

### Subset the data to only include purchases, not returns

In [31]:
poloData = poloData[poloData['SType'] == 'P']

In [32]:
poloData.shape

(5153583, 7)

### Subsetting SKUs

There are too many SKUs (142,108 unique SKUs) to do one hot encoding. So, I am going to use the 100 most common SKU values.

In [33]:
topSKU = pd.DataFrame(poloData['SKU'].value_counts())
topSKU.reset_index(level=0, inplace=True)
topSKU = topSKU.rename(columns = {'SKU':'count', 'index':'SKU'})

In [34]:
topSKU.head()

SKU  count
0  6486359   8611
1  6806359   6774
2  9352339   6597
3  9432339   6314
4  6246359   6309

In [35]:
top = topSKU[:100]

In [36]:
top = top.drop('count', axis = 1)

In [37]:
top.shape

(100, 1)

In [38]:
topData = poloData[poloData['SKU'].isin(top['SKU'])]

In [39]:
topData.head()

SKU  Store  Register  TranNum  SEQ       Date SType
5119856  439441    102       810      200    0 2005-05-24     P
5119857  439441    102       810     1900    0 2005-02-24     P
5119858  439441    102       810     2100    0 2005-03-03     P
5119859  439441    102       810     2800    0 2005-05-28     P
5119860  439441    102       810     4900    0 2005-03-04     P

In [40]:
topData.shape

(251364, 7)

## Defining a Market Basket

The next step is to organize items purchased together into market baskets. For our purposes, a unique order will have the same Store, Register, TranNum, SEQ, and Date. A variable called orderID will be created using Store, Register, TranNum, and SEQ. If two or more items have the same orderID and Date, then they are a part of the same market basket.

In [41]:
orderID = topData['Store']+topData['Register']+topData['TranNum']+topData['SEQ']+topData['Date']

In [42]:
data = pd.concat([topData,orderID], axis=1)
data = data.rename(columns = {0:'orderID'})

In [43]:
data.head()

SKU  Store  Register  TranNum  SEQ       Date SType  \
5119856  439441    102       810      200    0 2005-05-24     P   
5119857  439441    102       810     1900    0 2005-02-24     P   
5119858  439441    102       810     2100    0 2005-03-03     P   
5119859  439441    102       810     2800    0 2005-05-28     P   
5119860  439441    102       810     4900    0 2005-03-04     P   

                              orderID  
5119856 2005-05-24 00:00:00.000001112  
5119857 2005-02-24 00:00:00.000002812  
5119858 2005-03-03 00:00:00.000003012  
5119859 2005-05-28 00:00:00.000003712  
5119860 2005-03-04 00:00:00.000005812

In [44]:
len(data['SKU'].value_counts())

100

In [45]:
# Example problem, showing that there is only one unique Market Basket when sorting by date and orderID
ex1 = data.sort_values(by = ['orderID'])
ex1.head(10)

SKU  Store  Register  TranNum  SEQ       Date SType  \
64347068   5174599    507        90      400    0 2004-08-01     P   
17936272   1589441    203       440      500    0 2004-08-01     P   
12194832   1049441    203       440      500    0 2004-08-01     P   
68048414   5471167    102       810      300    0 2004-08-01     P   
77065993   6246359    704       390      400    0 2004-08-01     P   
24080575   2108810    503       350      800    0 2004-08-01     P   
67926303   5461167   1302       220      200    0 2004-08-01     P   
120572276  9972087   1007       121      600    0 2004-08-01     P   
24080652   2108810    807       230      800    0 2004-08-01     P   
5439014     469441   1302       220      400    0 2004-08-01     P   

                                orderID  
64347068  2004-08-01 00:00:00.000000997  
17936272  2004-08-01 00:00:00.000001143  
12194832  2004-08-01 00:00:00.000001143  
68048414  2004-08-01 00:00:00.000001212  
77065993  2004-08-01 00:00:00.000001494  
24080575  2004-08-01 00:00:00.000001653  
67926303  2004-08-01 00:00:00.000001722  
120572276 2004-08-01 00:00:00.000001728  
24080652  2004-08-01 00:00:00.000001837  
5439014   2004-08-01 00:00:00.000001922

In [46]:
onehot = pd.get_dummies(data['SKU'], prefix = 'SKU')

In [47]:
onehot.head()

SKU_439441  SKU_459441  SKU_469441  SKU_907523  SKU_936370  \
5119856           1           0           0           0           0   
5119857           1           0           0           0           0   
5119858           1           0           0           0           0   
5119859           1           0           0           0           0   
5119860           1           0           0           0           0   

         SKU_947523  SKU_992340  SKU_1049441  SKU_1069441  SKU_1072340  ...  \
5119856           0           0            0            0            0  ...   
5119857           0           0            0            0            0  ...   
5119858           0           0            0            0            0  ...   
5119859           0           0            0            0            0  ...   
5119860           0           0            0            0            0  ...   

         SKU_8932339  SKU_9002262  SKU_9042262  SKU_9272339  SKU_9352339  \
5119856            0            0            0            0            0   
5119857            0            0            0            0            0   
5119858            0            0            0            0            0   
5119859            0            0            0            0            0   
5119860            0            0            0            0            0   

         SKU_9432339  SKU_9502339  SKU_9883749  SKU_9972087  SKU_9993749  
5119856            0            0            0            0            0  
5119857            0            0            0            0            0  
5119858            0            0            0            0            0  
5119859            0            0            0            0            0  
5119860            0            0            0            0            0  

[5 rows x 100 columns]

In [48]:
onehot.shape

(251364, 100)

In [49]:
df = pd.concat([data,onehot],axis=1)

In [50]:
df.head()

SKU  Store  Register  TranNum  SEQ       Date SType  \
5119856  439441    102       810      200    0 2005-05-24     P   
5119857  439441    102       810     1900    0 2005-02-24     P   
5119858  439441    102       810     2100    0 2005-03-03     P   
5119859  439441    102       810     2800    0 2005-05-28     P   
5119860  439441    102       810     4900    0 2005-03-04     P   

                              orderID  SKU_439441  SKU_459441  ...  \
5119856 2005-05-24 00:00:00.000001112           1           0  ...   
5119857 2005-02-24 00:00:00.000002812           1           0  ...   
5119858 2005-03-03 00:00:00.000003012           1           0  ...   
5119859 2005-05-28 00:00:00.000003712           1           0  ...   
5119860 2005-03-04 00:00:00.000005812           1           0  ...   

         SKU_8932339  SKU_9002262  SKU_9042262  SKU_9272339  SKU_9352339  \
5119856            0            0            0            0            0   
5119857            0            0            0            0            0   
5119858            0            0            0            0            0   
5119859            0            0            0            0            0   
5119860            0            0            0            0            0   

         SKU_9432339  SKU_9502339  SKU_9883749  SKU_9972087  SKU_9993749  
5119856            0            0            0            0            0  
5119857            0            0            0            0            0  
5119858            0            0            0            0            0  
5119859            0            0            0            0            0  
5119860            0            0            0            0            0  

[5 rows x 108 columns]

In [51]:
df.drop(['SKU','Store','Register','TranNum','SEQ','Date','SType'],axis=1, inplace=True)

In [52]:
df.shape

(251364, 101)

In [53]:
df.head()

orderID  SKU_439441  SKU_459441  SKU_469441  \
5119856 2005-05-24 00:00:00.000001112           1           0           0   
5119857 2005-02-24 00:00:00.000002812           1           0           0   
5119858 2005-03-03 00:00:00.000003012           1           0           0   
5119859 2005-05-28 00:00:00.000003712           1           0           0   
5119860 2005-03-04 00:00:00.000005812           1           0           0   

         SKU_907523  SKU_936370  SKU_947523  SKU_992340  SKU_1049441  \
5119856           0           0           0           0            0   
5119857           0           0           0           0            0   
5119858           0           0           0           0            0   
5119859           0           0           0           0            0   
5119860           0           0           0           0            0   

         SKU_1069441  ...  SKU_8932339  SKU_9002262  SKU_9042262  SKU_9272339  \
5119856            0  ...            0            0            0            0   
5119857            0  ...            0            0            0            0   
5119858            0  ...            0            0            0            0   
5119859            0  ...            0            0            0            0   
5119860            0  ...            0            0            0            0   

         SKU_9352339  SKU_9432339  SKU_9502339  SKU_9883749  SKU_9972087  \
5119856            0            0            0            0            0   
5119857            0            0            0            0            0   
5119858            0            0            0            0            0   
5119859            0            0            0            0            0   
5119860            0            0            0            0            0   

         SKU_9993749  
5119856            0  
5119857            0  
5119858            0  
5119859            0  
5119860            0  

[5 rows x 101 columns]

In [54]:
finalData = df.groupby(['orderID']).sum()

In [55]:
finalData.head()

SKU_439441  SKU_459441  SKU_469441  SKU_907523  \
orderID                                                                         
2004-08-01 00:00:00.000000997           0           0           0           0   
2004-08-01 00:00:00.000001143           0           0           0           0   
2004-08-01 00:00:00.000001212           0           0           0           0   
2004-08-01 00:00:00.000001494           0           0           0           0   
2004-08-01 00:00:00.000001653           0           0           0           0   

                               SKU_936370  SKU_947523  SKU_992340  \
orderID                                                             
2004-08-01 00:00:00.000000997           0           0           0   
2004-08-01 00:00:00.000001143           0           0           0   
2004-08-01 00:00:00.000001212           0           0           0   
2004-08-01 00:00:00.000001494           0           0           0   
2004-08-01 00:00:00.000001653           0           0           0   

                               SKU_1049441  SKU_1069441  SKU_1072340  ...  \
orderID                                                               ...   
2004-08-01 00:00:00.000000997            0            0            0  ...   
2004-08-01 00:00:00.000001143            1            0            0  ...   
2004-08-01 00:00:00.000001212            0            0            0  ...   
2004-08-01 00:00:00.000001494            0            0            0  ...   
2004-08-01 00:00:00.000001653            0            0            0  ...   

                               SKU_8932339  SKU_9002262  SKU_9042262  \
orderID                                                                
2004-08-01 00:00:00.000000997            0            0            0   
2004-08-01 00:00:00.000001143            0            0            0   
2004-08-01 00:00:00.000001212            0            0            0   
2004-08-01 00:00:00.000001494            0            0            0   
2004-08-01 00:00:00.000001653            0            0            0   

                               SKU_9272339  SKU_9352339  SKU_9432339  \
orderID                                                                
2004-08-01 00:00:00.000000997            0            0            0   
2004-08-01 00:00:00.000001143            0            0            0   
2004-08-01 00:00:00.000001212            0            0            0   
2004-08-01 00:00:00.000001494            0            0            0   
2004-08-01 00:00:00.000001653            0            0            0   

                               SKU_9502339  SKU_9883749  SKU_9972087  \
orderID                                                                
2004-08-01 00:00:00.000000997            0            0            0   
2004-08-01 00:00:00.000001143            0            0            0   
2004-08-01 00:00:00.000001212            0            0            0   
2004-08-01 00:00:00.000001494            0            0            0   
2004-08-01 00:00:00.000001653            0            0            0   

                               SKU_9993749  
orderID                                     
2004-08-01 00:00:00.000000997            0  
2004-08-01 00:00:00.000001143            0  
2004-08-01 00:00:00.000001212            0  
2004-08-01 00:00:00.000001494            0  
2004-08-01 00:00:00.000001653            0  

[5 rows x 100 columns]

In [56]:
finalData.shape

(210929, 100)

In [57]:
final = finalData >= 1

In [58]:
a = final.sum()
b = a/886008

In [59]:
b = pd.DataFrame(b)
b.reset_index(level=0, inplace=True)
b = b.rename(columns = {0:'prob', 'index':'SKU'})

In [60]:
b.head()

SKU      prob
0  SKU_439441  0.002167
1  SKU_459441  0.002328
2  SKU_469441  0.001938
3  SKU_907523  0.002155
4  SKU_936370  0.002196

In [61]:
b.sort_values(by = 'prob',ascending=False)

SKU      prob
73  SKU_6486359  0.009614
76  SKU_6806359  0.007550
94  SKU_9352339  0.007144
69  SKU_6246359  0.007060
95  SKU_9432339  0.006795
9   SKU_1072340  0.004977
11  SKU_1152340  0.004972
93  SKU_9272339  0.004219
23  SKU_3269431  0.003604
39  SKU_5228351  0.003433
47  SKU_5461167  0.003340
77  SKU_6846359  0.003316
66  SKU_6149446  0.003306
96  SKU_9502339  0.003263
25  SKU_4343643  0.003114
18  SKU_2078810  0.003070
22  SKU_3249431  0.003042
6    SKU_992340  0.003028
79  SKU_6872339  0.003026
67  SKU_6169446  0.003019
92  SKU_9042262  0.002991
24  SKU_3279431  0.002955
41  SKU_5258351  0.002928
43  SKU_5321167  0.002905
64  SKU_6119446  0.002854
48  SKU_5471167  0.002830
46  SKU_5441167  0.002783
89  SKU_8852339  0.002754
42  SKU_5291167  0.002734
12  SKU_1232340  0.002716
..          ...       ...
35  SKU_5104599  0.002122
70  SKU_6433893  0.002116
68  SKU_6213596  0.002113
78  SKU_6863893  0.002109
84  SKU_8013602  0.002093
28  SKU_4403643  0.002088
17  SKU_1869441  0.002060
59  SKU_5753596  0.002051
81  SKU_7716294  0.002051
54  SKU_5689400  0.002049
98  SKU_9972087  0.002049
21  SKU_2967523  0.002041
75  SKU_6703893  0.002015
34  SKU_5024599  0.002003
61  SKU_5798351  0.001994
38  SKU_5174599  0.001993
32  SKU_4912330  0.001991
45  SKU_5363579  0.001984
65  SKU_6123596  0.001983
86  SKU_8366288  0.001981
53  SKU_5679400  0.001962
16  SKU_1707524  0.001957
14  SKU_1589441  0.001954
5    SKU_947523  0.001939
2    SKU_469441  0.001938
33  SKU_4994599  0.001911
19  SKU_2108810  0.001906
10  SKU_1079441  0.001893
37  SKU_5154599  0.001893
20  SKU_2672329  0.001457

[100 rows x 2 columns]

In [62]:
frequentItemsets = apriori(final, min_support=0.0001, use_colnames=True)

In [63]:
frequentItemsets

support                                 itemsets
0    0.009103                             (SKU_439441)
1    0.009781                             (SKU_459441)
2    0.008140                             (SKU_469441)
3    0.009050                             (SKU_907523)
4    0.009226                             (SKU_936370)
5    0.008145                             (SKU_947523)
6    0.012720                             (SKU_992340)
7    0.009193                            (SKU_1049441)
8    0.010018                            (SKU_1069441)
9    0.020908                            (SKU_1072340)
10   0.007951                            (SKU_1079441)
11   0.020884                            (SKU_1152340)
12   0.011407                            (SKU_1232340)
13   0.009297                            (SKU_1571243)
14   0.008207                            (SKU_1589441)
15   0.009823                            (SKU_1677524)
16   0.008221                            (SKU_1707524)
17   0.008652                            (SKU_1869441)
18   0.012895                            (SKU_2078810)
19   0.008007                            (SKU_2108810)
20   0.006121                            (SKU_2672329)
21   0.008572                            (SKU_2967523)
22   0.012777                            (SKU_3249431)
23   0.015138                            (SKU_3269431)
24   0.012412                            (SKU_3279431)
25   0.013080                            (SKU_4343643)
26   0.009269                            (SKU_4353643)
27   0.009719                            (SKU_4373643)
28   0.008771                            (SKU_4403643)
29   0.010719                            (SKU_4421180)
..        ...                                      ...
296  0.000180               (SKU_9042262, SKU_9432339)
297  0.000218               (SKU_9272339, SKU_9352339)
298  0.000104               (SKU_9272339, SKU_9432339)
299  0.000299               (SKU_9432339, SKU_9352339)
300  0.000119               (SKU_9502339, SKU_9432339)
301  0.000474   (SKU_439441, SKU_1589441, SKU_1049441)
302  0.000493   (SKU_1069441, SKU_1869441, SKU_459441)
303  0.000247  (SKU_6447365, SKU_9352339, SKU_1072340)
304  0.000204  (SKU_1152340, SKU_6457365, SKU_9432339)
305  0.000109  (SKU_1152340, SKU_9432339, SKU_8613189)
306  0.000104  (SKU_1571243, SKU_2078810, SKU_5901243)
307  0.000114  (SKU_1571243, SKU_2108810, SKU_4912330)
308  0.000104  (SKU_2108810, SKU_1571243, SKU_9972087)
309  0.000251  (SKU_1571243, SKU_5901243, SKU_4912330)
310  0.000147  (SKU_1571243, SKU_9972087, SKU_5901243)
311  0.000161  (SKU_2108810, SKU_2078810, SKU_9972087)
312  0.000104  (SKU_9972087, SKU_2078810, SKU_5901243)
313  0.000109  (SKU_2108810, SKU_5901243, SKU_4912330)
314  0.000123  (SKU_2108810, SKU_9972087, SKU_4912330)
315  0.000427  (SKU_6149446, SKU_5709431, SKU_3269431)
316  0.000450  (SKU_3279431, SKU_6169446, SKU_5719431)
317  0.000109  (SKU_5228351, SKU_4898351, SKU_5549233)
318  0.000209  (SKU_5228351, SKU_4898351, SKU_5748351)
319  0.000119  (SKU_9972087, SKU_5901243, SKU_4912330)
320  0.000119  (SKU_5228351, SKU_5549233, SKU_5748351)
321  0.000190  (SKU_6486359, SKU_6246359, SKU_6806359)
322  0.000133  (SKU_6486359, SKU_6246359, SKU_6846359)
323  0.000119  (SKU_6806359, SKU_6246359, SKU_6846359)
324  0.000119  (SKU_6457365, SKU_9432339, SKU_8613189)
325  0.000119  (SKU_6486359, SKU_6846359, SKU_6806359)

[326 rows x 2 columns]

In [64]:
rules = association_rules(frequentItemsets, metric="lift", min_threshold=1)

In [65]:
rules

antecedents                 consequents  \
0                  (SKU_439441)               (SKU_1049441)   
1                 (SKU_1049441)                (SKU_439441)   
2                  (SKU_439441)               (SKU_1069441)   
3                 (SKU_1069441)                (SKU_439441)   
4                  (SKU_439441)               (SKU_1589441)   
5                 (SKU_1589441)                (SKU_439441)   
6                  (SKU_439441)               (SKU_3249431)   
7                 (SKU_3249431)                (SKU_439441)   
8                  (SKU_439441)               (SKU_6119446)   
9                 (SKU_6119446)                (SKU_439441)   
10                (SKU_1069441)                (SKU_459441)   
11                 (SKU_459441)               (SKU_1069441)   
12                (SKU_1869441)                (SKU_459441)   
13                 (SKU_459441)               (SKU_1869441)   
14                (SKU_3269431)                (SKU_459441)   
15                 (SKU_459441)               (SKU_3269431)   
16                (SKU_4421180)                (SKU_459441)   
17                 (SKU_459441)               (SKU_4421180)   
18                (SKU_6149446)                (SKU_459441)   
19                 (SKU_459441)               (SKU_6149446)   
20                 (SKU_469441)               (SKU_1079441)   
21                (SKU_1079441)                (SKU_469441)   
22                 (SKU_469441)               (SKU_3279431)   
23                (SKU_3279431)                (SKU_469441)   
24                 (SKU_469441)               (SKU_6169446)   
25                (SKU_6169446)                (SKU_469441)   
26                (SKU_9272339)                (SKU_992340)   
27                 (SKU_992340)               (SKU_9272339)   
28                (SKU_1589441)               (SKU_1049441)   
29                (SKU_1049441)               (SKU_1589441)   
..                          ...                         ...   
408  (SKU_6486359, SKU_6246359)               (SKU_6806359)   
409  (SKU_6486359, SKU_6806359)               (SKU_6246359)   
410  (SKU_6246359, SKU_6806359)               (SKU_6486359)   
411               (SKU_6486359)  (SKU_6246359, SKU_6806359)   
412               (SKU_6246359)  (SKU_6486359, SKU_6806359)   
413               (SKU_6806359)  (SKU_6486359, SKU_6246359)   
414  (SKU_6486359, SKU_6246359)               (SKU_6846359)   
415  (SKU_6486359, SKU_6846359)               (SKU_6246359)   
416  (SKU_6246359, SKU_6846359)               (SKU_6486359)   
417               (SKU_6486359)  (SKU_6246359, SKU_6846359)   
418               (SKU_6246359)  (SKU_6486359, SKU_6846359)   
419               (SKU_6846359)  (SKU_6486359, SKU_6246359)   
420  (SKU_6806359, SKU_6246359)               (SKU_6846359)   
421  (SKU_6806359, SKU_6846359)               (SKU_6246359)   
422  (SKU_6246359, SKU_6846359)               (SKU_6806359)   
423               (SKU_6806359)  (SKU_6246359, SKU_6846359)   
424               (SKU_6246359)  (SKU_6806359, SKU_6846359)   
425               (SKU_6846359)  (SKU_6806359, SKU_6246359)   
426  (SKU_6457365, SKU_9432339)               (SKU_8613189)   
427  (SKU_6457365, SKU_8613189)               (SKU_9432339)   
428  (SKU_9432339, SKU_8613189)               (SKU_6457365)   
429               (SKU_6457365)  (SKU_9432339, SKU_8613189)   
430               (SKU_9432339)  (SKU_6457365, SKU_8613189)   
431               (SKU_8613189)  (SKU_6457365, SKU_9432339)   
432  (SKU_6486359, SKU_6846359)               (SKU_6806359)   
433  (SKU_6486359, SKU_6806359)               (SKU_6846359)   
434  (SKU_6806359, SKU_6846359)               (SKU_6486359)   
435               (SKU_6486359)  (SKU_6806359, SKU_6846359)   
436               (SKU_6846359)  (SKU_6486359, SKU_6806359)   
437               (SKU_6806359)  (SKU_6486359, SKU_6846359)   

     antecedent support  consequent support   support  confidence       lift  \
0              0.009103          

In [66]:
rules.describe()

antecedent support  consequent support     support  confidence  \
count          438.000000          438.000000  438.000000  438.000000   
mean             0.010345            0.010345    0.000471    0.073210   
std              0.007187            0.007187    0.000512    0.078111   
min              0.000318            0.000318    0.000104    0.002935   
25%              0.008363            0.008363    0.000129    0.015443   
50%              0.009297            0.009297    0.000251    0.038598   
75%              0.011837            0.011837    0.000531    0.107968   
max              0.040383            0.040383    0.002423    0.344828   

             lift    leverage  conviction  
count  438.000000  438.000000  438.000000  
mean     9.391543    0.000370    1.076387  
std      9.109040    0.000463    0.102819  
min      1.015415    0.000002    1.000153  
25%      2.189526    0.000100    1.008498  
50%      5.583817    0.000177    1.028905  
75%     14.065761    0.000450    1.106857  
max     37.511159    0.002212    1.512285

In [67]:
rules[ rules['lift'] >= 10 ]

antecedents                 consequents  \
0                  (SKU_439441)               (SKU_1049441)   
1                 (SKU_1049441)                (SKU_439441)   
4                  (SKU_439441)               (SKU_1589441)   
5                 (SKU_1589441)                (SKU_439441)   
10                (SKU_1069441)                (SKU_459441)   
11                 (SKU_459441)               (SKU_1069441)   
12                (SKU_1869441)                (SKU_459441)   
13                 (SKU_459441)               (SKU_1869441)   
20                 (SKU_469441)               (SKU_1079441)   
21                (SKU_1079441)                (SKU_469441)   
28                (SKU_1589441)               (SKU_1049441)   
29                (SKU_1049441)               (SKU_1589441)   
34                (SKU_1069441)               (SKU_1869441)   
35                (SKU_1869441)               (SKU_1069441)   
68                (SKU_1571243)               (SKU_5901243)   
69                (SKU_5901243)               (SKU_1571243)   
90                (SKU_2108810)               (SKU_9972087)   
91                (SKU_9972087)               (SKU_2108810)   
92                (SKU_6119446)               (SKU_3249431)   
93                (SKU_3249431)               (SKU_6119446)   
94                (SKU_5709431)               (SKU_3269431)   
95                (SKU_3269431)               (SKU_5709431)   
96                (SKU_6149446)               (SKU_3269431)   
97                (SKU_3269431)               (SKU_6149446)   
100               (SKU_3279431)               (SKU_5719431)   
101               (SKU_5719431)               (SKU_3279431)   
102               (SKU_3279431)               (SKU_6169446)   
103               (SKU_6169446)               (SKU_3279431)   
156               (SKU_5331167)               (SKU_5471167)   
157               (SKU_5471167)               (SKU_5331167)   
..                          ...                         ...   
382               (SKU_6169446)  (SKU_3279431, SKU_5719431)   
383               (SKU_5719431)  (SKU_3279431, SKU_6169446)   
384  (SKU_5228351, SKU_4898351)               (SKU_5549233)   
385  (SKU_5228351, SKU_5549233)               (SKU_4898351)   
386  (SKU_4898351, SKU_5549233)               (SKU_5228351)   
387               (SKU_5228351)  (SKU_4898351, SKU_5549233)   
388               (SKU_4898351)  (SKU_5228351, SKU_5549233)   
389               (SKU_5549233)  (SKU_5228351, SKU_4898351)   
390  (SKU_5228351, SKU_4898351)               (SKU_5748351)   
391  (SKU_5228351, SKU_5748351)               (SKU_4898351)   
392  (SKU_4898351, SKU_5748351)               (SKU_5228351)   
393               (SKU_5228351)  (SKU_4898351, SKU_5748351)   
394               (SKU_4898351)  (SKU_5228351, SKU_5748351)   
395               (SKU_5748351)  (SKU_5228351, SKU_4898351)   
396  (SKU_9972087, SKU_5901243)               (SKU_4912330)   
397  (SKU_9972087, SKU_4912330)               (SKU_5901243)   
398  (SKU_5901243, SKU_4912330)               (SKU_9972087)   
399               (SKU_9972087)  (SKU_5901243, SKU_4912330)   
400               (SKU_5901243)  (SKU_9972087, SKU_4912330)   
401               (SKU_4912330)  (SKU_9972087, SKU_5901243)   
402  (SKU_5228351, SKU_5549233)               (SKU_5748351)   
403  (SKU_5228351, SKU_5748351)               (SKU_5549233)   
404  (SKU_5549233, SKU_5748351)               (SKU_5228351)   
405               (SKU_5228351)  (SKU_5549233, SKU_5748351)   
406               (SKU_5549233)  (SKU_5228351, SKU_5748351)   
407               (SKU_5748351)  (SKU_5228351, SKU_5549233)   
427  (SKU_6457365, SKU_8613189)               (SKU_9432339)   
428  (SKU_9432339, SKU_8613189)               (SKU_6457365)   
429               (SKU_6457365)  (SKU_9432339, SKU_8613189)   
430               (SKU_9432339)  (SKU_6457365, SKU_8613189)   

     antecedent support  consequent support   support  confidence       lift  \
0              0.009103          

In [68]:
rules[ rules['lift'] >= 10 ].describe()

antecedent support  consequent support     support  confidence  \
count          146.000000          146.000000  146.000000  146.000000   
mean             0.007010            0.007010    0.000593    0.131224   
std              0.005382            0.005382    0.000676    0.098502   
min              0.000318            0.000318    0.000104    0.004153   
25%              0.001025            0.001025    0.000119    0.021670   
50%              0.008605            0.008605    0.000209    0.139735   
75%              0.009805            0.009805    0.001005    0.198432   
max              0.030010            0.030010    0.002423    0.344828   

             lift    leverage  conviction  
count  146.000000  146.000000  146.000000  
mean    20.411443    0.000556    1.157795  
std      7.303378    0.000630    0.132768  
min     10.197491    0.000095    1.003813  
25%     14.276482    0.000110    1.020857  
50%     19.500359    0.000200    1.152392  
75%     25.308696    0.000936    1.235138  
max     37.511159    0.002212    1.512285

In [69]:
new_rules = rules[(rules['lift'] >= 15) & (rules['confidence'] >= 0.25)]

In [70]:
len(rules[(rules['lift'] >= 15) & (rules['confidence'] >= 0.25)])

15

In [71]:
new_rules

antecedents    consequents  antecedent support  \
288   (SKU_439441, SKU_1589441)  (SKU_1049441)            0.001375   
289   (SKU_439441, SKU_1049441)  (SKU_1589441)            0.001721   
290  (SKU_1589441, SKU_1049441)   (SKU_439441)            0.001664   
294  (SKU_1069441, SKU_1869441)   (SKU_459441)            0.001930   
295   (SKU_1069441, SKU_459441)  (SKU_1869441)            0.001911   
296   (SKU_1869441, SKU_459441)  (SKU_1069441)            0.001550   
318  (SKU_1571243, SKU_2078810)  (SKU_5901243)            0.000318   
337  (SKU_1571243, SKU_4912330)  (SKU_5901243)            0.000749   
338  (SKU_5901243, SKU_4912330)  (SKU_1571243)            0.000773   
342  (SKU_1571243, SKU_9972087)  (SKU_5901243)            0.000498   
367  (SKU_2108810, SKU_4912330)  (SKU_9972087)            0.000484   
374  (SKU_5709431, SKU_3269431)  (SKU_6149446)            0.001683   
379  (SKU_3279431, SKU_5719431)  (SKU_6169446)            0.001474   
386  (SKU_4898351, SKU_5549233)  (SKU_5228351)            0.000346   
404  (SKU_5549233, SKU_5748351)  (SKU_5228351)            0.000460   

     consequent support   support  confidence       lift  leverage  conviction  
288            0.009193  0.000474    0.344828  37.511159  0.000461    1.512285  
289            0.008207  0.000474    0.275482  33.568551  0.000460    1.368901  
290            0.009103  0.000474    0.284900  31.298819  0.000459    1.385677  
294            0.009781  0.000493    0.255528  26.126185  0.000474    1.330097  
295            0.008652  0.000493    0.258065  29.826460  0.000477    1.336164  
296            0.010018  0.000493    0.318043  31.748439  0.000478    1.451678  
318            0.009273  0.000104    0.328358  35.409135  0.000101    1.475082  
337            0.009273  0.000251    0.335443  36.173141  0.000244    1.490808  
338            0.009297  0.000251    0.325153  34.974134  0.000244    1.468042  
342            0.009273  0.000147    0.295238  31.837565  0.000142    1.405761  
367            0.008605  0.000123    0.254902  29.623259  0.000119    1.330557  
374            0.013886  0.000427    0.253521  18.257070  0.000403    1.321020  
379            0.012682  0.000450    0.305466  24.086612  0.000432    1.421555  
386            0.014422  0.000109    0.315068  21.846510  0.000104    1.438944  
404            0.014422  0.000119    0.257732  17.870856  0.000112    1.327793

In [72]:
new_rules.describe()

antecedent support  consequent support    support  confidence  \
count           15.000000           15.000000  15.000000   15.000000   
mean             0.001129            0.010406   0.000326    0.293849   
std              0.000619            0.002228   0.000169    0.033044   
min              0.000318            0.008207   0.000104    0.253521   
25%              0.000491            0.009148   0.000135    0.257898   
50%              0.001375            0.009273   0.000427    0.295238   
75%              0.001674            0.011350   0.000474    0.321598   
max              0.001930            0.014422   0.000493    0.344828   

            lift   leverage  conviction  
count  15.000000  15.000000   15.000000  
mean   29.343860   0.000314    1.404291  
std     6.350264   0.000163    0.066514  
min    17.870856   0.000101    1.321020  
25%    25.106399   0.000131    1.333361  
50%    31.298819   0.000403    1.405761  
75%    34.271343   0.000461    1.459860  
max    37.511159   0.000478    1.512285

In [73]:
#new_rules.to_excel("rules.xlsx")

In [74]:
rules[rules['lift'] >= 37.511159]

antecedents                consequents  antecedent support  \
288  (SKU_439441, SKU_1589441)              (SKU_1049441)            0.001375   
293              (SKU_1049441)  (SKU_439441, SKU_1589441)            0.009193   

     consequent support   support  confidence       lift  leverage  conviction  
288            0.009193  0.000474    0.344828  37.511159  0.000461    1.512285  
293            0.001375  0.000474    0.051573  37.511159  0.000461    1.052928

In [75]:
rules[rules['lift'] >= 30]

antecedents                 consequents  \
288   (SKU_439441, SKU_1589441)               (SKU_1049441)   
289   (SKU_439441, SKU_1049441)               (SKU_1589441)   
290  (SKU_1589441, SKU_1049441)                (SKU_439441)   
291                (SKU_439441)  (SKU_1589441, SKU_1049441)   
292               (SKU_1589441)   (SKU_439441, SKU_1049441)   
293               (SKU_1049441)   (SKU_439441, SKU_1589441)   
296   (SKU_1869441, SKU_459441)               (SKU_1069441)   
297               (SKU_1069441)   (SKU_1869441, SKU_459441)   
318  (SKU_1571243, SKU_2078810)               (SKU_5901243)   
323               (SKU_5901243)  (SKU_1571243, SKU_2078810)   
337  (SKU_1571243, SKU_4912330)               (SKU_5901243)   
338  (SKU_5901243, SKU_4912330)               (SKU_1571243)   
339               (SKU_1571243)  (SKU_5901243, SKU_4912330)   
340               (SKU_5901243)  (SKU_1571243, SKU_4912330)   
342  (SKU_1571243, SKU_9972087)               (SKU_5901243)   
347               (SKU_5901243)  (SKU_1571243, SKU_9972087)   

     antecedent support  consequent support   support  confidence       lift  \
288            0.001375            0.009193  0.000474    0.344828  37.511159   
289            0.001721            0.008207  0.000474    0.275482  33.568551   
290            0.001664            0.009103  0.000474    0.284900  31.298819   
291            0.009103            0.001664  0.000474    0.052083  31.298819   
292            0.008207            0.001721  0.000474    0.057770  33.568551   
293            0.009193            0.001375  0.000474    0.051573  37.511159   
296            0.001550            0.010018  0.000493    0.318043  31.748439   
297            0.010018            0.001550  0.000493    0.049219  31.748439   
318            0.000318            0.009273  0.000104    0.328358  35.409135   
323            0.009273            0.000318  0.000104    0.011247  35.409135   
337            0.000749            0.009273  0.000251    0.335443  36.173141   
338            0.000773            0.009297  0.000251    0.325153  34.974134   
339            0.009297            0.000773  0.000251    0.027027  34.974134   
340            0.009273            0.000749  0.000251    0.027096  36.173141   
342            0.000498            0.009273  0.000147    0.295238  31.837565   
347            0.009273            0.000498  0.000147    0.015849  31.837565   

     leverage  conviction  
288  0.000461    1.512285  
289  0.000460    1.368901  
290  0.000459    1.385677  
291  0.000459    1.053190  
292  0.000460    1.059486  
293  0.000461    1.052928  
296  0.000478    1.451678  
297  0.000478    1.050137  
318  0.000101    1.475082  
323  0.000101    1.011054  
337  0.000244    1.490808  
338  0.000244    1.468042  
339  0.000244    1.026984  
340  0.000244    1.027081  
342  0.000142    1.405761  
347  0.000142    1.015598

In [76]:
skuinfo[skuinfo['SKU'].isin([1049441,1589441,439441])]

SKU  Dept ClassID           UPC         Style         Color  \
68584    439441  4505     404  400009441043  2BAO  581965  NAVY           
163747  1049441  4505     404  400009441104  7BAO  581972  UNFR KHAKI     
248103  1589441  4505     404  400009441158  3BAO  581965  SAND           

              Size Packsize   Vendor      Brand  
68584   34                1  5745232  POLO FAS   
163747  34                1  5745232  POLO FAS   
248103  34                1  5745232  POLO FAS

In [77]:
skuinfo[skuinfo['SKU'].isin([1869441,459441,1069441])]

SKU  Dept ClassID           UPC         Style         Color  \
71637    459441  4505     404  400009441045  2BAO  581965  NAVY           
166877  1069441  4505     404  400009441106  7BAO  581972  UNFR KHAKI     
291253  1869441  4505     404  400009441186  3BAO  581965  SAND           

              Size Packsize   Vendor      Brand  
71637   36                1  5745232  POLO FAS   
166877  36                1  5745232  POLO FAS   
291253  36                1  5745232  POLO FAS

In [78]:
skuinfo[skuinfo['SKU'].isin([2078810,1571243,5901243,4912330,9972087,2108810])]

SKU  Dept ClassID           UPC         Style         Color  \
245379   1571243  4505     302  400001243157  3VPP  650185  SPA BLUE       
323397   2078810  4505     302  400008810207  5VPP  651649  BSC WHITE      
328070   2108810  4505     302  400008810210  3VPP  651649  NUBUCK         
769457   4912330  4505     302  400002330491  3VPP  656523  VALOR RED      
925021   5901243  4505     302  400001243590  7VPP  656475  OASIS YELL     
1560073  9972087  4505     302  400002087997  2VPP  650100  RELAY BLUE     

               Size Packsize   Vendor      Brand  
245379   ONE               1  5715232  POLO FAS   
323397   ONE               1  5715232  POLO FAS   
328070   ONE               1  5715232  POLO FAS   
769457   ONE               1  5715232  POLO FAS   
925021   ONE               1  5715232  POLO FAS   
1560073  ONE               1  5715232  POLO FAS

In [79]:
skuinfo[skuinfo['SKU'].isin([5709431,3269431,6149446])]

SKU  Dept ClassID           UPC         Style         Color  \
510957  3269431  4505     404  400009431326  3BRG  581965  SAND           
894494  5709431  4505     404  400009431570  2BRG  581965  NAVY           
963876  6149446  4505     404  400009446614  7BRG  581972  UNFR KHAKI     

              Size Packsize   Vendor      Brand  
510957  36                1  5745232  POLO FAS   
894494  36                1  5745232  POLO FAS   
963876  36                1  5745232  POLO FAS

In [80]:
skuinfo[skuinfo['SKU'].isin([3279431,5719431,6169446])]

SKU  Dept ClassID           UPC         Style         Color  \
512447  3279431  4505     404  400009431327  3BRG  581965  SAND           
896023  5719431  4505     404  400009431571  2BRG  581965  NAVY           
966969  6169446  4505     404  400009446616  7BRG  581972  UNFR KHAKI     

              Size Packsize   Vendor      Brand  
512447  38                1  5745232  POLO FAS   
896023  38                1  5745232  POLO FAS   
966969  38                1  5745232  POLO FAS

In [81]:
skuinfo[skuinfo['SKU'].isin([4898351,5228351,5549233,5748351])]

SKU  Dept ClassID           UPC         Style         Color  \
767035  4898351  4505     107  400008351489  2AEB  793365  CLSC BLUE      
818845  5228351  4505     107  400008351522  5AEB  793365  CLSC WHITE     
869386  5549233  4505     107  400009233554  6AEB  790003  W/W INDIGO     
900529  5748351  4505     107  400008351574  7AEB  793365  BLU/WHT STR    

              Size Packsize   Vendor      Brand  
767035  L                 1  5745232  POLO FAS   
818845  L                 1  5745232  POLO FAS   
869386  L                 1  5745232  POLO FAS   
900529  L                 1  5745232  POLO FAS

### Top100 used as a table in the report

In [82]:
top.head()

SKU
0  6486359
1  6806359
2  9352339
3  9432339
4  6246359

In [83]:
top100 = skuinfo[skuinfo['SKU'].isin(top['SKU'])]

In [84]:
top100 = top100.reset_index()

In [85]:
top100 = top100.drop(columns = ['index','Dept','ClassID','UPC','Packsize','Vendor'])

In [86]:
top100.head()

SKU         Style         Color        Size      Brand
0  439441  2BAO  581965  NAVY          34          POLO FAS 
1  459441  2BAO  581965  NAVY          36          POLO FAS 
2  469441  2BAO  581965  NAVY          38          POLO FAS 
3  907523  2ANJA 796964  WHITE         L           POLO FAS 
4  936370  2A    796812  BLUE/GREEN    L           POLO FAS

In [87]:
#top100.to_excel("output.xlsx")